In [1]:
import sys
sys.path.append("../")


In [6]:
from sqlmodel import Session, select
from models.vault_performance import VaultPerformance
import pandas as pd
from core.db import engine

session = Session(engine)

query = """
SELECT datetime,total_locked_value
FROM public.vault_performance
WHERE vault_id = '65f75bd7-a2d2-4764-ae31-78e4bb132c62'
ORDER BY datetime ASC
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql(query, session.bind)
df["datetime"] = pd.to_datetime(df["datetime"])
if df["datetime"].dt.tz is None:
        df["datetime"] = df["datetime"].dt.tz_localize("UTC")
else:
        df["datetime"] = df["datetime"].dt.tz_convert("UTC")

df["datetime"] = df["datetime"].dt.floor("D")
df.set_index("datetime", inplace=True)

daily_df = df.resample("D").first()
daily_df.ffill(inplace=True)
daily_df.reset_index(inplace=True)
# Display the resulting DataFrame
daily_df

,datetime,total_locked_value
0,2024-06-04 00:00:00+00:00,150.115912
1,2024-06-05 00:00:00+00:00,150.083810
2,2024-06-06 00:00:00+00:00,150.070816
3,2024-06-07 00:00:00+00:00,716.565137
4,2024-06-08 00:00:00+00:00,717.030704
...,...,...
179,2024-11-30 00:00:00+00:00,89599.225107
180,2024-12-01 00:00:00+00:00,90027.086283
181,2024-12-02 00:00:00+00:00,97560.577767
182,2024-12-03 00:00:00+00:00,98245.903035


In [15]:
ALLOCATION_RATIO: float = 1 / 2
AE_USD = 0.08/365
LST_YEILD = 0.036/365

In [5]:
from datetime import datetime, timedelta

def get_daily_funding_rate_df(file_path: str):
    df = pd.read_csv(file_path)

    # Convert 'datetime' to UTC and normalize to the start of the day
    df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")

    # Handle invalid or missing datetime values
    if df["datetime"].isna().any():
        raise ValueError("Invalid datetime format detected in the 'datetime' column")

    # Ensure all datetimes are in UTC
    if df["datetime"].dt.tz is None:
        df["datetime"] = df["datetime"].dt.tz_localize("UTC")
    else:
        df["datetime"] = df["datetime"].dt.tz_convert("UTC")

    # Resample to daily frequency and calculate mean funding rate
    daily_avg = (
        df.set_index("datetime").resample("D")["funding_rate"].mean().reset_index()
    )

    # Rename column to make it more descriptive
    daily_avg.rename(
        columns={"datetime": "date", "funding_rate": "average_funding_rate"},
        inplace=True,
    )
    return daily_avg


def get_funding_history(
    funding_histories_avg_df: pd.DataFrame, target_datetime: datetime
) -> float:
    filtered_row = funding_histories_avg_df.loc[
        funding_histories_avg_df["date"] == target_datetime
    ]
    if not filtered_row.empty:
        return filtered_row.iloc[0]["average_funding_rate"]
    return 0

In [12]:
CSV_PATH = {
    "BSX": "../data/funding_history_bsx.csv",
    "AEVO": "../data/funding_history_aevo.csv",
    "HYPERLIQUID": "../data/funding_history_hyperliquid.csv",
    "GOLDLINK": "../data/funding_history_goldlink.csv",
}


hyperliquid_funding_history_df = get_daily_funding_rate_df("../data/funding_history_hyperliquid.csv")
hyperliquid_funding_history_df

,date,average_funding_rate
0,2024-04-05 00:00:00+00:00,0.000029
1,2024-04-06 00:00:00+00:00,0.000033
2,2024-04-07 00:00:00+00:00,0.000015
3,2024-04-08 00:00:00+00:00,0.000020
4,2024-04-09 00:00:00+00:00,0.000020
...,...,...
239,2024-11-30 00:00:00+00:00,0.000081
240,2024-12-01 00:00:00+00:00,0.000038
241,2024-12-02 00:00:00+00:00,0.000054
242,2024-12-03 00:00:00+00:00,0.000043


In [16]:
from datetime import datetime, timedelta, timezone

from bg_tasks.update_yield_vault_performance_init import get_tvl_from_prev_date

funding_rate_dt = []
hyperliquid_funding_history_df = get_daily_funding_rate_df(CSV_PATH["HYPERLIQUID"])
aevo_funding_history_df = get_daily_funding_rate_df(CSV_PATH["AEVO"])


# foreach row in df1, calculate the roi
for i, row in daily_df.iterrows():
    if i == 0:
        continue
    date = row["datetime"]
    date_move_vault = datetime(2024, 11, 11, tzinfo=timezone.utc)
    funding_history = (
            get_funding_history(hyperliquid_funding_history_df, date)
            if date >= date_move_vault
            else get_funding_history(aevo_funding_history_df, date)
        )

    prev_tvl = get_tvl_from_prev_date(daily_df, i - 1)
    funding_value = funding_history * ALLOCATION_RATIO * 24 * prev_tvl
    ae_usd_value = AE_USD * ALLOCATION_RATIO * prev_tvl
    lst_yield_value = LST_YEILD * ALLOCATION_RATIO * prev_tvl
    yield_data = funding_value + ae_usd_value + lst_yield_value

    total_locked_value= float(daily_df.iloc[i-1]["total_locked_value"])

    funding_rate_dt.append(
        {
            "datetime": date,
            "prev_tvl":prev_tvl,
            "tvl": row["total_locked_value"],
            "funding_rate": funding_history,
            "LST_YEILD": lst_yield_value,
            "AE_USD": ae_usd_value,
            "funding": funding_value,
            "yield_data": yield_data,
        }
    )

funding_rate_df = pd.DataFrame(funding_rate_dt)

# In ra DataFrame
print(funding_rate_df)

                     datetime      prev_tvl            tvl  funding_rate  \
0   2024-06-05 00:00:00+00:00    150.115912     150.083810      0.000035   
1   2024-06-06 00:00:00+00:00    150.083810     150.070816      0.000036   
2   2024-06-07 00:00:00+00:00    150.070816     716.565137      0.000049   
3   2024-06-08 00:00:00+00:00    716.565137     717.030704      0.000017   
4   2024-06-09 00:00:00+00:00    717.030704     717.133939     -0.000002   
..                        ...           ...            ...           ...   
178 2024-11-30 00:00:00+00:00  89486.665723   89599.225107      0.000081   
179 2024-12-01 00:00:00+00:00  89599.225107   90027.086283      0.000038   
180 2024-12-02 00:00:00+00:00  90027.086283   97560.577767      0.000054   
181 2024-12-03 00:00:00+00:00  97560.577767   98245.903035      0.000043   
182 2024-12-04 00:00:00+00:00  98245.903035  121359.179874      0.000064   

     LST_YEILD     AE_USD    funding  yield_data  
0     0.007403   0.016451   0.063049

In [18]:
funding_rate_df.to_csv("funding_rate_data.csv", index=False)